In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [4]:
accuracy_vsp = []
accuracy = []

for i in range(20):
    X_train, X_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, label_material, label_visiprog, \
                     test_size=0.7, stratify=label_material)

    model = LFDA()
    model.fit(X_train, Y_mat_train)
    X_mat_train = model.transform(X_train)
    X_mat_test = model.transform(X_test)
        
    res = leave_one_sample_out(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test)
    accuracy.append(res['accuracy'])

    model_vsp = LFDA_VISIPROG()
    model_vsp.fit(X_train, Y_VSP_train)
    X_vsp_train = model_vsp.transform(X_train)
    X_vsp_test = model_vsp.transform(X_test)
    res_vsp = leave_one_sample_out(X_vsp_train, Y_mat_train, X_vsp_test, Y_mat_test)
    accuracy_vsp.append(res_vsp['accuracy'])
        


INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:vis

In [5]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy})
display(df)
display(df.mean())

,material,visiprog
0,0.960512,0.894063
1,0.960240,0.908497
2,0.949346,0.923203
3,0.964869,0.918845
4,0.962963,0.913943
5,0.964869,0.893519
6,0.963780,0.907680
7,0.967048,0.901961
8,0.961057,0.922113
9,0.960512,0.928105


material    0.96186
visiprog    0.90802
dtype: float64

In [6]:
print(Y_mat_test)

[55 18  8 ...,  8 44 53]
